# Création d'utilisateurs ADS via un CSV
Ce notebook sert à créer des utilisateurs ADS via l'API `POST /api/alpha/ads/token/`>

Il est à modifier si besoin en fonction du format de CSV en entrée.

In [ ]:
import csv
import json
import re
import requests
import os

csv_path_input = os.environ['INPUT_CSV_FILE']
csv_path_output = os.environ['OUTPUT_CSV_FILE']
base_url = 'https://rnb-api.beta.gouv.fr/api/alpha/'
token_superuser = os.environ['SUPERUSER_TOKEN']

In [ ]:
# Open the input csv file
with open(csv_path_input, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')

    # Skip headers
    next(csvreader, None)

    # Build our model from the CSV file. CSV format:
    # ['4z0bXY', 'lWj245', '2024-06-17 15:25:29', 'some@gmail.com', 'Test org', '75056;93048;93006']
    all_users = []
    for row in csvreader:
        delimiter = ' '
        if ',' in row[5]:
            delimiter = ','
        elif ';' in row[5]:
            delimiter = ';'

        codes_insee = [element.strip() for element in re.split(delimiter, row[5]) if element.strip()]

        all_users.append({
            "username": row[3],
            "email": row[3],
            "organization_name": row[4],
            "organization_managed_cities": codes_insee
        })

    # Create users using API
    response = requests.post(base_url + 'ads/token/',
                             data=json.dumps(all_users),
                             headers={'Authorization': 'Token ' + token_superuser})

    all_created_users = response.json()['created_users']

    # Write the result in the CSV output file
    with open(csv_path_output, 'w', encoding='UTF8', newline='') as csvfile_output:
        writer = csv.writer(csvfile_output)
        writer.writerow(['username', 'organization_name', 'email', 'password', 'token'])
        for user in all_created_users:
            writer.writerow(list(user.values()))
